This is the Project on Clustering of *_Toronto Data _*

In [257]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize

In [258]:
!pip install folium

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


# Data Collection Notebook [Using Pandas]

The dataframe will consist of three columns: **PostalCode, Borough, and Neighborhood**

In [259]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r  = requests.get(url)
data = r.text
soup = BeautifulSoup(data)
table = soup.find('table', attrs={'class': 'wikitable sortable'})

In [260]:
Neighbourhood = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighbourhood'])
for tr in table.find_all('tr')[1:]:
    tds = tr.find_all('td')
    row = [tr.text.strip() for tr in tds if tr.text.strip()]
    Neighbourhood = Neighbourhood.append({'PostalCode': row[0],
                                        'Borough': row[1],
                                        'Neighbourhood': row[2]}, ignore_index = True)
    
Neighbourhood.head(180)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [261]:
Exclude = Neighbourhood[Neighbourhood['Borough'] != 'Not assigned']
Exclude.head(180)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [262]:
Neighbourhood_df = Exclude.groupby(['PostalCode', 'Borough']).agg(', '.join)
Neighbourhood_df.reset_index(inplace = True)


In [263]:
U= Neighbourhood_df['Neighbourhood'] == 'Not assigned'
Neighbourhood_df.loc[U,'Neighbourhood'] = Neighbourhood_df.loc[U,'Borough']
Neighbourhood_df.loc[U]

,PostalCode,Borough,Neighbourhood


In [264]:
Neighbourhood_df.head(20)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


# Merging with the Geo-Coordinates

In [265]:
Lat_df = pd.read_csv('http://cocl.us/Geospatial_data')
Lat_df.rename(columns = {'Postal Code': 'PostalCode'}, inplace = True)
Lat_df.head()


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [266]:

neigh_merge_df = pd.merge(Neighbourhood_df, Lat_df, on='PostalCode', how='left')
neigh_merge_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [267]:
latitude = 43.7032
longitude = -79.3832
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.7032, -79.3832.


In [268]:
import folium as fo

In [269]:


toronto_map = fo.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(neigh_merge_df['Latitude'], neigh_merge_df['Longitude'], neigh_merge_df['Neighbourhood']):
    label = fo.Popup(label, parse_html=True)
    fo.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='cyan',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

In [270]:
CLIENT_ID = '1YZJO3UY3Y1IXCYCE3WLWZWKEOU3GEK2MBP15OSG0QH5GKR4'
CLIENT_SECRET = 'VGX2KLUEJPDOLLWC043AZKN0FRF2RVMN20M5OIG5UYGSGYZF'
VERSION = '20180605'

print('MY CLIENT_ID: ' + CLIENT_ID)
print('MY CLIENT_SECRET:' + CLIENT_SECRET)

MY CLIENT_ID: 1YZJO3UY3Y1IXCYCE3WLWZWKEOU3GEK2MBP15OSG0QH5GKR4
MY CLIENT_SECRET:VGX2KLUEJPDOLLWC043AZKN0FRF2RVMN20M5OIG5UYGSGYZF


In [271]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
      
        results = requests.get(url).json()["response"]['groups'][0]['items']
      
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [272]:
toronto_venues = getNearbyVenues(names=neigh_merge_df['Neighbourhood'],latitudes=neigh_merge_df['Latitude'],longitudes=neigh_merge_df['Longitude'])

In [273]:
print("Our venues dataset includes", toronto_venues.shape[0], 'venues, each has', toronto_venues.shape[1], 'attributes')
print('There are', len(toronto_venues['Venue Category'].unique()), 'categories in total')
toronto_venues.head()

Our venues dataset includes 2141 venues, each has 7 attributes
There are 273 categories in total


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


# **This is a feature vector**

In [274]:
onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
onehot.insert(0, 'NeighbourhoodName', toronto_venues['Neighbourhood'])

In [275]:
grouped = onehot.groupby('NeighbourhoodName').sum().reset_index()
grouped.head()

,NeighbourhoodName,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Bathurst Manor, Wilson Heights, Downsview North",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bayview Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Bedford Park, Lawrence Manor East",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [276]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [277]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = grouped['NeighbourhoodName']

for ind in np.arange(grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head(95)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Discount Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Gym,Sandwich Place,Coffee Shop,Pub,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop,Doner Restaurant,Department Store,Drugstore
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Fried Chicken Joint,Chinese Restaurant,Bridal Shop,Sandwich Place,Diner,Restaurant,Middle Eastern Restaurant,Supermarket,Mobile Phone Shop,Sushi Restaurant,Deli / Bodega,Frozen Yogurt Shop,Ice Cream Shop
3,Bayview Village,Japanese Restaurant,Café,Chinese Restaurant,Bank,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Greek Restaurant,Thai Restaurant,Locksmith,Liquor Store,Comfort Food Restaurant,Juice Bar,Butcher,Café,Restaurant,Sushi Restaurant,Pub,Indian Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale East",Ramen Restaurant,Coffee Shop,Pizza Place,Sandwich Place,Japanese Restaurant,Café,Sushi Restaurant,Movie Theater,Hotel,Shopping Mall,Grocery Store,Electronics Store,Lounge,Discount Store,Juice Bar
91,"Willowdale, Willowdale West",Pizza Place,Grocery Store,Coffee Shop,Butcher,Pharmacy,Gym,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
92,Woburn,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Diner,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant
93,Woodbine Heights,Skating Rink,Park,Curling Ice,Athletics & Sports,Dance Studio,Intersection,Beer Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Discount Store,Event Space,Drugstore,Donut Shop


In [295]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [296]:
kclusters = 3
X = grouped.drop('NeighbourhoodName', 1).astype(float)
X = StandardScaler().fit_transform(X)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(X)
yhat = kmeans.labels_

In [301]:

toronto_grouped_clustering = grouped.drop('NeighbourhoodName', 1)
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kMeans.labels_)
kMeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)

toronto_merged = toronto_venues.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on = 'Neighbourhood')
toronto_merged.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant,2,Fast Food Restaurant,Dumpling Restaurant,...,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Health & Beauty Service,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar,2,Construction & Landscaping,Bar,...,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Discount Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping,2,Construction & Landscaping,Bar,...,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Discount Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank,2,Breakfast Spot,Restaurant,...,Intersection,Bank,Mexican Restaurant,Yoga Studio,Doner Restaurant,Distribution Center,Dog Run,Drugstore,Donut Shop,Dumpling Restaurant
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store,2,Breakfast Spot,Restaurant,...,Intersection,Bank,Mexican Restaurant,Yoga Studio,Doner Restaurant,Distribution Center,Dog Run,Drugstore,Donut Shop,Dumpling Restaurant


In [306]:
# create map
map_clusters = fo.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Neighbourhood Latitude'],toronto_merged['Neighbourhood Longitude'], toronto_merged['Neighbourhood'],toronto_merged['Cluster Labels']):
    label = fo.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    fo.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [305]:
toronto_merged[toronto_merged['Cluster Labels'] == 0]

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
95,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store,0,Clothing Store,Coffee Shop,...,Juice Bar,Cosmetics Shop,Bank,Bakery,Food Court,Restaurant,Shoe Store,Pharmacy,Salon / Barbershop,Intersection
96,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,CF Fairview Mall,43.777981,-79.344397,Shopping Mall,0,Clothing Store,Coffee Shop,...,Juice Bar,Cosmetics Shop,Bank,Bakery,Food Court,Restaurant,Shoe Store,Pharmacy,Salon / Barbershop,Intersection
97,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Moxie's Classic Grill,43.777779,-79.343185,American Restaurant,0,Clothing Store,Coffee Shop,...,Juice Bar,Cosmetics Shop,Bank,Bakery,Food Court,Restaurant,Shoe Store,Pharmacy,Salon / Barbershop,Intersection
98,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,SilverCity,43.778681,-79.344085,Movie Theater,0,Clothing Store,Coffee Shop,...,Juice Bar,Cosmetics Shop,Bank,Bakery,Food Court,Restaurant,Shoe Store,Pharmacy,Salon / Barbershop,Intersection
99,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Apple Fairview,43.777883,-79.343789,Electronics Store,0,Clothing Store,Coffee Shop,...,Juice Bar,Cosmetics Shop,Bank,Bakery,Food Court,Restaurant,Shoe Store,Pharmacy,Salon / Barbershop,Intersection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2028,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Tim Hortons,43.658906,-79.388696,Coffee Shop,0,Coffee Shop,Yoga Studio,...,Sandwich Place,Restaurant,Café,Chinese Restaurant,Portuguese Restaurant,College Auditorium,Gym,Park,Music Venue,Creperie
2029,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Understudy Café at Gerstein,43.662308,-79.394098,College Cafeteria,0,Coffee Shop,Yoga Studio,...,Sandwich Place,Restaurant,Café,Chinese Restaurant,Portuguese Restaurant,College Auditorium,Gym,Park,Music Venue,Creperie
2030,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Gallery Grill,43.663841,-79.394309,Restaurant,0,Coffee Shop,Yoga Studio,...,Sandwich Place,Restaurant,Café,Chinese Restaurant,Portuguese Restaurant,College Auditorium,Gym,Park,Music Venue,Creperie
2031,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Hart House Music Room,43.663758,-79.395027,Music Venue,0,Coffee Shop,Yoga Studio,...,Sandwich Place,Restaurant,Café,Chinese Restaurant,Portuguese Restaurant,College Auditorium,Gym,Park,Music Venue,Creperie


In [307]:
toronto_merged[toronto_merged['Cluster Labels'] == 1]

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
717,"Garden District, Ryerson",43.657162,-79.378937,UNIQLO ユニクロ,43.655910,-79.380641,Clothing Store,1,Clothing Store,Coffee Shop,...,Bubble Tea Shop,Pizza Place,Furniture / Home Store,Bookstore,Fast Food Restaurant,Ramen Restaurant,Middle Eastern Restaurant,Theater,Hotel,Lingerie Store
718,"Garden District, Ryerson",43.657162,-79.378937,Yonge-Dundas Square,43.656054,-79.380495,Plaza,1,Clothing Store,Coffee Shop,...,Bubble Tea Shop,Pizza Place,Furniture / Home Store,Bookstore,Fast Food Restaurant,Ramen Restaurant,Middle Eastern Restaurant,Theater,Hotel,Lingerie Store
719,"Garden District, Ryerson",43.657162,-79.378937,Page One Cafe,43.657772,-79.376073,Café,1,Clothing Store,Coffee Shop,...,Bubble Tea Shop,Pizza Place,Furniture / Home Store,Bookstore,Fast Food Restaurant,Ramen Restaurant,Middle Eastern Restaurant,Theater,Hotel,Lingerie Store
720,"Garden District, Ryerson",43.657162,-79.378937,Silver Snail Comics,43.657031,-79.381403,Comic Shop,1,Clothing Store,Coffee Shop,...,Bubble Tea Shop,Pizza Place,Furniture / Home Store,Bookstore,Fast Food Restaurant,Ramen Restaurant,Middle Eastern Restaurant,Theater,Hotel,Lingerie Store
721,"Garden District, Ryerson",43.657162,-79.378937,Burrito Boyz,43.656265,-79.378343,Burrito Place,1,Clothing Store,Coffee Shop,...,Bubble Tea Shop,Pizza Place,Furniture / Home Store,Bookstore,Fast Food Restaurant,Ramen Restaurant,Middle Eastern Restaurant,Theater,Hotel,Lingerie Store
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1787,"First Canadian Place, Underground city",43.648429,-79.382280,Leña,43.651722,-79.379205,Latin American Restaurant,1,Coffee Shop,Café,...,Restaurant,Steakhouse,American Restaurant,Salad Place,Asian Restaurant,Seafood Restaurant,Bar,Beer Bar,Gastropub,Deli / Bodega
1788,"First Canadian Place, Underground city",43.648429,-79.382280,Prairie Girl Bakery,43.648332,-79.382305,Cupcake Shop,1,Coffee Shop,Café,...,Restaurant,Steakhouse,American Restaurant,Salad Place,Asian Restaurant,Seafood Restaurant,Bar,Beer Bar,Gastropub,Deli / Bodega
1789,"First Canadian Place, Underground city",43.648429,-79.382280,Copacabana Grilled Brazilian,43.648333,-79.388151,Brazilian Restaurant,1,Coffee Shop,Café,...,Restaurant,Steakhouse,American Restaurant,Salad Place,Asian Restaurant,Seafood Restaurant,Bar,Beer Bar,Gastropub,Deli / Bodega
1790,"First Canadian Place, Underground city",43.648429,-79.382280,Piper's Gastropub,43.645468,-79.381779,Cocktail Bar,1,Coffee Shop,Café,...,Restaurant,Steakhouse,American Restaurant,Salad Place,Asian Restaurant,Seafood Restaurant,Bar,Beer Bar,Gastropub,Deli / Bodega


In [308]:
toronto_merged[toronto_merged['Cluster Labels'] == 2]

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant,2,Fast Food Restaurant,Dumpling Restaurant,...,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Health & Beauty Service,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar,2,Construction & Landscaping,Bar,...,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Discount Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping,2,Construction & Landscaping,Bar,...,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Discount Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank,2,Breakfast Spot,Restaurant,...,Intersection,Bank,Mexican Restaurant,Yoga Studio,Doner Restaurant,Distribution Center,Dog Run,Drugstore,Donut Shop,Dumpling Restaurant
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store,2,Breakfast Spot,Restaurant,...,Intersection,Bank,Mexican Restaurant,Yoga Studio,Doner Restaurant,Distribution Center,Dog Run,Drugstore,Donut Shop,Dumpling Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,LCBO,43.741508,-79.584501,Liquor Store,2,Pizza Place,Grocery Store,...,Liquor Store,Beer Store,Fast Food Restaurant,Gluten-free Restaurant,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
2137,"Northwest, West Humber - Clairville",43.706748,-79.594054,Economy Rent A Car,43.708471,-79.589943,Rental Car Location,2,Drugstore,Garden Center,...,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant
2138,"Northwest, West Humber - Clairville",43.706748,-79.594054,Logistics Distribution,43.707554,-79.589252,Bar,2,Drugstore,Garden Center,...,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant
2139,"Northwest, West Humber - Clairville",43.706748,-79.594054,Saand Rexdale,43.705072,-79.598725,Drugstore,2,Drugstore,Garden Center,...,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant


**The first cluster shows posh neighbourhoods popular amongst young people and old people as it has recreational facilities**
**The second cluster is popular among the tourists as it has eloquent restaurants and theater,etc**

**The last cluster is popular among poorer people and immigrants, probably Middle Eastern and African people**